In [46]:
from typing import Optional
from dataclasses import dataclass
from datetime import datetime
import requests

@dataclass
class Pronostico:
    fecha: str
    temperatura_max: float
    temperatura_min: float
    lluvia_mm: float
    descripcion: str
    indice_uv: Optional[int] = None
    
    
    iconos_tiempo = {
            "Despejado": "☀️",
            "Mayormente despejado": "🌤️",
            "Parcialmente nublado": "⛅",
            "Nublado": "☁️",
            "Niebla": "🌫️",
            "Niebla con escarcha": "❄️🌫️",
            "Llovizna ligera": "🌦️",
            "Llovizna moderada": "🌧️",
            "Llovizna densa": "🌧️",
            "Lluvia ligera": "🌦️",
            "Lluvia moderada": "🌧️",
            "Lluvia fuerte": "🌧️💧",
            "Nieve ligera": "🌨️",
            "Nieve moderada": "❄️",
            "Nieve fuerte": "❄️❄️",
            "Chubascos ligeros": "🌦️",
            "Chubascos moderados": "🌧️",
            "Chubascos fuertes": "⛈️",
            "Tormenta": "⛈️⚡",
            "Tormenta con granizo leve": "⛈️🌨️",
            "Tormenta con granizo fuerte": "⛈️🌨️",
            "Desconocido": "❓"
    }

    def __str__(self) -> str:
        condiciones_str = f"{self.iconos_tiempo.get(self.descripcion, '❓')} {self.descripcion.capitalize()}"
        
        temp_max_str = f"{self.temperatura_max}°C"
        if self.temperatura_max >= 30:
            temp_max_str = f"🥵 {temp_max_str}"
        elif self.temperatura_max >= 25:
            temp_max_str = f"🌡️ {temp_max_str}"

        temperatura_str = f"{self.temperatura_min}°C – {temp_max_str}"

        lluvia_str = f"💧 {self.lluvia_mm:.1f} mm" if self.lluvia_mm > 0.4 else ''
        uv_str = f"UV: {self.indice_uv:.1f}" if self.indice_uv is not None else ''

        partes = [condiciones_str, temperatura_str, lluvia_str, uv_str]
        return ', '.join(p for p in partes if p)

    def to_list(self) -> list:
        return [
            self.fecha,
             self.iconos_tiempo.get(self.descripcion, '❓') + self.descripcion,
            f"{self.temperatura_min}°C",
            f"{self.temperatura_max}°C" + ('🌡️' if self.temperatura_max >= 25 else ''),
            f"{self.lluvia_mm:.1f} mm" if self.lluvia_mm > 0.4 else '',
            str(self.indice_uv) if self.indice_uv is not None else ''
        ]

@dataclass
class Ciudad:
    nombre: str
    latitud: float
    longitud: float
    pais: str = "Francia"
    pronosticos: list[Pronostico] = None

    def coordenadas(self) -> tuple[float, float]:
        return (self.latitud, self.longitud)

    def agregar_pronostico(self, pronostico: Pronostico) -> None:
        if self.pronosticos is None:
            self.pronosticos = []
        self.pronosticos.append(pronostico)

    def __str__(self) -> str:
        return f"{self.nombre} ({self.latitud}, {self.longitud})"

def interpretar_weathercode(code: int) -> str:
    codigos = {
        0: "Despejado",
        1: "Mayormente despejado",
        2: "Parcialmente nublado",
        3: "Nublado",
        45: "Niebla",
        48: "Niebla con escarcha",
        51: "Llovizna ligera",
        53: "Llovizna moderada",
        55: "Llovizna densa",
        61: "Lluvia ligera",
        63: "Lluvia moderada",
        65: "Lluvia fuerte",
        71: "Nieve ligera",
        73: "Nieve moderada",
        75: "Nieve fuerte",
        80: "Chubascos ligeros",
        81: "Chubascos moderados",
        82: "Chubascos fuertes",
        95: "Tormenta",
        96: "Tormenta con granizo leve",
        99: "Tormenta con granizo fuerte"
    }
    return codigos.get(code, "Desconocido")

def obtener_pronostico(ciudad: Ciudad, fecha: datetime) -> Optional[Pronostico]:
    """Consulta la previsión meteorológica para una ciudad y una fecha concreta."""
    base_url = "https://api.open-meteo.com/v1/forecast"
    lat, lon = ciudad.coordenadas()
    fecha_str = fecha.strftime("%Y-%m-%d")

    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": [
            "temperature_2m_max",
            "temperature_2m_min",
            "precipitation_sum",
            "uv_index_max",
            "weathercode"
        ],
        "timezone": "Europe/Paris",
        "start_date": fecha_str,
        "end_date": fecha_str
    }

    response = requests.get(base_url, params=params)
    if not response.ok:
        print(f"Error al obtener datos para {ciudad.nombre} el {fecha_str}")
        return None

    data = response.json().get("daily", {})
    try:
        idx = 0  # Solo una fecha
        pronostico = Pronostico(
            fecha=fecha_str,
            temperatura_max=data["temperature_2m_max"][idx],
            temperatura_min=data["temperature_2m_min"][idx],
            lluvia_mm=data["precipitation_sum"][idx],
            descripcion=interpretar_weathercode(data["weathercode"][idx]),
            indice_uv=data.get("uv_index_max", [None])[idx]
        )
        return pronostico
    except (KeyError, IndexError) as e:
        print(f"Datos incompletos para {ciudad.nombre} el {fecha_str}: {e}")
        return None


In [37]:
# Creamos las ciudades
from typing import Dict, List


ciudades_data = {
    "Angers": (47.4784, -0.5632),
    "Saumur": (47.2597, -0.0786),
    "Chinon": (47.1674, 0.2416),
    "Langeais": (47.3253, 0.4033),
    "Azay-le-Rideau": (47.2628, 0.4669),
    "Villandry": (47.3384, 0.5150),
    "Loches": (47.1290, 0.9996),
    "Amboise": (47.4136, 0.9824),
    "Tours": (47.3941, 0.6848),
    "Montrésor": (47.0739, 1.2046),
    "Chenonceaux": (47.3292, 1.0718),
    "Cheverny": (47.4967, 1.4582),
    "Blois": (47.5861, 1.3350),
    "Orléans": (47.9022, 1.9044),
    "Chambord": (47.6649, 1.5163),
    "Bilbao": (43.2630, -2.9350),
}

# Creamos objetos Ciudad
ciudades: Dict[str, Ciudad] = {
    nombre: Ciudad(nombre, lat, lon)
    for nombre, (lat, lon) in ciudades_data.items()
}

# Diccionario {fecha: List[Ciudad]} con el itinerario
itinerario: Dict[str, List[Ciudad]] = {
    "2025-07-25": [ciudades["Angers"], ciudades["Saumur"], ciudades["Chinon"]],
    "2025-07-26": [ciudades["Angers"], ciudades["Langeais"], ciudades["Azay-le-Rideau"], ciudades["Villandry"], ciudades["Loches"]],
    "2025-07-27": [ciudades["Loches"], ciudades["Amboise"], ciudades["Tours"]],
    "2025-07-28": [ciudades["Loches"], ciudades["Montrésor"], ciudades["Chenonceaux"]],
    "2025-07-29": [ciudades["Loches"], ciudades["Cheverny"], ciudades["Blois"]],
    "2025-07-30": [ciudades["Blois"], ciudades["Orléans"]],
    "2025-07-31": [ciudades["Blois"], ciudades["Chambord"]],
    "2025-08-01": [ciudades["Blois"], ciudades["Bilbao"]],
}


In [47]:
for fecha_str, lista_ciudades in itinerario.items():
    fecha_dt = datetime.strptime(fecha_str, "%Y-%m-%d")
    print(f"\n📅 Pronóstico para el día {fecha_str}:")
    for ciudad in lista_ciudades:
        pron = obtener_pronostico(ciudad, fecha_dt)
        if pron:
            ciudad.agregar_pronostico(pron)
            print(f" - {ciudad.nombre}: {pron}")
        else:
            print(f" - {ciudad.nombre}: No se pudo obtener el pronóstico.")


📅 Pronóstico para el día 2025-07-25:
 - Angers: ☁️ Nublado, 16.4°C – 24.8°C, UV: 6.8
 - Saumur: ☁️ Nublado, 16.2°C – 24.8°C, UV: 6.5
 - Chinon: ☁️ Nublado, 14.8°C – 24.9°C, UV: 7.0

📅 Pronóstico para el día 2025-07-26:
 - Angers: ☁️ Nublado, 14.5°C – 🌡️ 25.4°C, UV: 7.2
 - Langeais: ☁️ Nublado, 14.8°C – 24.4°C, UV: 7.2
 - Azay-le-Rideau: ☁️ Nublado, 15.0°C – 24.2°C, UV: 7.2
 - Villandry: 🌫️ Niebla, 14.8°C – 24.1°C, UV: 7.2
 - Loches: ☁️ Nublado, 15.6°C – 23.8°C, UV: 7.2

📅 Pronóstico para el día 2025-07-27:
 - Loches: 🌦️ Chubascos ligeros, 16.3°C – 19.9°C, 💧 18.9 mm, UV: 6.8
 - Amboise: 🌦️ Lluvia ligera, 16.6°C – 19.7°C, 💧 19.8 mm, UV: 6.6
 - Tours: 🌦️ Chubascos ligeros, 16.5°C – 20.2°C, 💧 19.8 mm, UV: 6.5

📅 Pronóstico para el día 2025-07-28:
 - Loches: ☁️ Nublado, 13.1°C – 23.1°C, UV: 6.5
 - Montrésor: ⛅ Parcialmente nublado, 12.8°C – 22.4°C, UV: 6.0
 - Chenonceaux: ☁️ Nublado, 13.3°C – 23.0°C, UV: 6.3

📅 Pronóstico para el día 2025-07-29:
 - Loches: ☁️ Nublado, 14.0°C – 24.2°C, UV: 

In [39]:
import pandas as pd

# Suponemos que ya tienes el dict itinerario y los objetos Ciudad con sus pronósticos

# Cabeceras para el DataFrame
columnas = ["Cuidad", "Fecha", "Estado", "Temp. min", "Temp. max", "Lluvia", "Índice UV"]

filas = []
for fecha_str, lista_ciudades in itinerario.items():
    for ciudad in lista_ciudades:
        # Buscar el pronóstico para la fecha
        pron = next((p for p in (ciudad.pronosticos or []) if p.fecha == fecha_str), None)
        if pron:
            filas.append([ciudad.nombre] + pron.to_list())

df = pd.DataFrame(filas, columns=columnas)

df

,Cuidad,Fecha,Estado,Temp. min,Temp. max,Lluvia,Índice UV
0,Angers,2025-07-25,☁️Nublado,16.4°C,24.8°C,,6.8
1,Saumur,2025-07-25,☁️Nublado,16.2°C,24.8°C,,6.55
2,Chinon,2025-07-25,☁️Nublado,14.8°C,24.9°C,,6.95
3,Angers,2025-07-26,☁️Nublado,14.5°C,25.4°C,,7.2
4,Langeais,2025-07-26,☁️Nublado,14.8°C,24.4°C,,7.15
5,Azay-le-Rideau,2025-07-26,☁️Nublado,15.0°C,24.2°C,,7.2
6,Villandry,2025-07-26,🌫️Niebla,14.8°C,24.1°C,,7.15
7,Loches,2025-07-26,☁️Nublado,15.6°C,23.8°C,,7.2
8,Loches,2025-07-27,🌦️Chubascos ligeros,16.3°C,19.9°C,💧 18.9 mm,6.85
9,Amboise,2025-07-27,🌦️Lluvia ligera,16.6°C,19.7°C,💧 19.8 mm,6.6
